In [5]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

In [7]:
!pip install requests
!pip install selenium
!pip install beautifulsoup4
!pip install html5lib
!pip install lxml

In [163]:
teams_url = "https://www.oddsportal.com/basketball/usa/nba-2022-2023/results/#/page/1/"
path = "//a[normalize-space()='Next']"

In [135]:
script = """
        els = document.getElementsByClassName('flex items-center justify-start');
        for (var i = els.length - 1; i >= 0; i--) {el = els[i].nextSibling;
        	if(el && el.parentElement) {el.parentElement.removeChild(el);
        		els[i].parentElement.removeChild(els[i]);	
        	}
        }
    """ 

In [109]:
from selenium import webdriver
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import time
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchWindowException, TimeoutException
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [243]:
driver = webdriver.Chrome()
driver.get(teams_url)
count = 0
dfs = []
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    try:
        sleep(5)
        driver.execute_script(script)
        sleep(5)

        count += 1
        html = driver.page_source
        
        with open(f"odds/page_{count}.html","w+") as f:
            f.write(html)
        
        button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, path)))
        driver.execute_script("arguments[0].click();", button)
        
    except (TimeoutException, NoSuchWindowException):
        break

driver.quit()

In [289]:
team_path = "eventRow flex w-full flex-col text-xs"
date_path = "text-black-main font-main w-full truncate text-xs font-normal leading-5"

In [460]:
import pandas as pd
from bs4 import BeautifulSoup

def create_dataframe_from_html(filename):
  data_list = []
  with open(filename) as f:
    page = f.read()

  soup = BeautifulSoup(page, "html.parser")
  rows = len(soup.find_all("div", class_= team_path))

  for num in range(rows):

    if num == 0:
        data_list.append(read_first("odds/page_1.html"))
        date = soup.find_all("div", class_= team_path)[0].find("div", class_= date_path).text
        data_list[num][0] = date
        continue
    
    teams = soup.find_all("div", class_= team_path)[num]

    if (teams.find("div", class_= date_path) == None):
        date = date
    else:
        date = teams.find("div", class_= date_path).text
    home = teams.find_all("p")[1].text.strip()
    away = teams.find_all("p")[2].text.strip()
    home_odds = teams.find_all("p")[3].text.strip()
    away_odds = teams.find_all("p")[4].text.strip()
    
    if home == "OT":
        home = teams.find_all("p")[2].text.strip()
        away = teams.find_all("p")[3].text.strip()
        home_odds = teams.find_all("p")[5].text.strip()
        away_odds = teams.find_all("p")[6].text.strip()

    data_list.append([date, home, away, home_odds, away_odds])
    num += 1

  df = pd.DataFrame(data_list, columns=['date', 'home', 'away', 'home_odds', 'away_odds'])
  return df

In [456]:
def read_first(filename2):
    with open(filename2) as f:
        page1 = f.read()
    soup1 = BeautifulSoup(page1, "html.parser")
    teams1 = soup1.find_all("div", class_= team_path)[0]
    home = teams1.find_all("p")[1].text.strip()
    away = teams1.find_all("p")[2].text.strip()
    home_odds = teams1.find_all("p")[3].text.strip()
    away_odds = teams1.find_all("p")[4].text.strip()
    return(["", home, away, home_odds, away_odds])

In [462]:
filename1 = "odds/page_2.html"
df = create_dataframe_from_html(filename1)

In [464]:
df

,date,home,away,home_odds,away_odds
0,12 Jun 2023 - Play Offs,Denver Nuggets,Miami Heat,-323,+264
1,09 Jun 2023 - Play Offs,Miami Heat,Denver Nuggets,+125,-147
2,07 Jun 2023 - Play Offs,Miami Heat,Denver Nuggets,+136,-161
3,04 Jun 2023 - Play Offs,Denver Nuggets,Miami Heat,-323,+263
4,01 Jun 2023 - Play Offs,Denver Nuggets,Miami Heat,-357,+284
5,29 May 2023 - Play Offs,Boston Celtics,Miami Heat,-303,+244
6,27 May 2023 - Play Offs,Miami Heat,Boston Celtics,+106,-125
7,25 May 2023 - Play Offs,Boston Celtics,Miami Heat,-312,+252
8,23 May 2023 - Play Offs,Miami Heat,Boston Celtics,-114,-103
9,22 May 2023 - Play Offs,Los Angeles Lakers,Denver Nuggets,-164,+138
